# Overview #
The goal of this personal project is to implement the AlphaZero algorithm to familiarize myself with Machine Learning and AI. I have learned a lot in my Machine Learning Class at Carnegie Mellon University, and now it is time for me to try out more complex algorithms such as AlphaZero! This is the last step of another course I have taken online on Free Code Camp called "AlphaZero from Scratch".
Although AlphaZero was initially implemented for Go! and Chess, I unfortunately do not have the computing power to train the model on such complex games. I therefore decided to implement it on the tic-tac-toe game as proposed in the online course. However, this change only reflects how powerful and versatile the AlphaZero algorithm actually is. 

# Step 1 # : Define the Tic-Tac-Toe rules
The first step is to set the rules of the tic-tac-toe game for our program to follow. 

In [1]:
import numpy as np
#for those of you who might be interested in my version of numpy :
np.__version__

'1.24.3'

In [2]:
#We first start by defining a class TicTacToe :
class TicTacToe:
    #Initialize the game board parameters
    def __init__(self):
        self.row_count = 3 
        self.column_count = 3
        self.action_size = self.row_count * self.column_count # action is an integer between 0 and 8 that indicates which case we decided to play in

    # Initialize the board to an array of zeros
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))     

    def get_next_state(self, state, action, player):        
        row = action // self.row_count # integer division to get the row coordinate of the action
        column = action % self.column_count
        state[row, column] = player #player is either X or O
        return state

    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8) # first we flatten the state and we check if the state is equal to 0. The type uint allow us to get from type boolean to type unsigned int

    def check_win(self, state, action):
        row = action // self.row_count
        column = action % self.column_count
        player = state[row, column]
        #check all 4 win conditions
        return (
            np.sum(state[row, :]) == player * self.column_count # 3 at horizontal
            or np.sum(state[:, column]) == player * self.row_count # 3 at vertical
            or np.sum(np.diag(state)) == player * self.column_count # 3 at diagonal
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.column_count # 3 at antidiagonal
        )

    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True   # someone has won, we return true
        elif np.sum(self.get_valid_moves(state)) == 0:
            return 0, True # no one has won and there are no valid moves left, we return true since the game has ended
        return 0, False # no one has won and the game can still be played

    def get_opponent(self, player):
        return -player

In [3]:
TTT = TicTacToe()
player = 1

state = TTT.get_initial_state()

while True:
    print(state)
    valid_moves = TTT.get_valid_moves(state)
    print("valid moves", [i for i in range(TTT.action_size) if valid_moves[i] == 1])
    action = int(input(f"{player}:"))

    if valid_moves[action] == 0:
        print("action not valid")
        continue

    state = TTT.get_next_state(state, action, player)

    value, is_terminal = TTT.get_value_and_terminated(state, action)

    if is_terminal:
        print(state)
        if value == 1:
            print(player, "won")
        else:
            print("draw")
        break

    player = TTT.get_opponent(player) # reiterates but for the second player
            

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves [0, 1, 2, 3, 4, 5, 6, 7, 8]


1: 1


[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves [0, 2, 3, 4, 5, 6, 7, 8]


-1: 2


[[ 0.  1. -1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
valid moves [0, 3, 4, 5, 6, 7, 8]


...

[[ 0.  1. -1.]
 [ 1. -1.  0.]
 [ 0.  0.  0.]]
valid moves [0, 5, 6, 7, 8]


1: 5


[[ 0.  1. -1.]
 [ 1. -1.  1.]
 [ 0.  0.  0.]]
valid moves [0, 6, 7, 8]


-1: 6


[[ 0.  1. -1.]
 [ 1. -1.  1.]
 [-1.  0.  0.]]
-1 won


# Implementing the Monte-Carlo Tree Search (MTCS) #
Now that we have implemented our TicTacToe game rules, it is time for us to write the code for the MTCS. This MTCS algorithm is fundamental as it will help us to define the best next move for the current state. For more information about the MTCS, see this link : https://builtin.com/machine-learning/monte-carlo-tree-search

In [ ]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None): #parent=None and action_taken=None account for the fact that the root node has no parents
        #we define the overall structure of the Monte-Carlo tree
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken                            

        self.children = []
        self.expandable_moves = game.get_valid_moves(state)              
        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expandable(self):    #we only select the nodes that are fully extended for our MTCS. To determine if node is fully extended, we count the number of expandable moves (ie. possible moves) and check if it equal to 0
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0      #additionally, we also consider that node is fully expanded if there is only one leaf node resulting from it (or a leaf branch)

    def select(self):
        
class MTCS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state): #we care about the number of times a node has been visited, therefore we shall return the value visit_counts  
        #we define the root node
        root = Node(self.game, self.args, state)

        for search in range(self.args['num_searches']):
            #we perform respectively selection, expansion, simulation, then backpropagation in this order
            node = root

            #selection :
            while node.is_fully_expanded():
                node = node.select() 


            #expansion :


            #simulation :


            
            #backpropagation :